**Vincular trabalhos do Pinto a estados e gerar taxas específicas por estado**

In [ ]:
!pip install python-docx
from docx import Document
from google.colab import files
import pandas as pd
from docx.shared import Inches

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 3.5 MB/s eta 0:00:00


In [ ]:

# ============================================
# Análise 2 - Taxas específicas por UF (Pinto)
# e extrapolação por rodovias (pav / npav / porte)
# ============================================



# -------------------------------------------------
# 1. Carregar bases
#    Ajuste os caminhos se necessário no Colab
# -------------------------------------------------

# Rodovias por UF (mesma tabela da Análise 1)
df_rodovias = pd.read_csv('/content/tabela_pav_npav_numerico.csv', sep=',', on_bad_lines='warn')

# Taxas por espécie e porte (df1 da Task 918)
df_taxas_porte = pd.read_csv('/content/taxas_pinto_especie_e_porte.csv', sep=',', on_bad_lines='warn')

# Tabela suplementar de Pinto com UFs (arquivo que você anexou)
df_pinto_ufs = pd.read_csv('/content/pinto_table_s1_with_ufs_final.csv', sep=',', on_bad_lines='warn')

# (Opcional) dar uma olhada nas colunas
print("Rodovias - colunas:", df_rodovias.columns.tolist())
print("Taxas por porte - colunas:", df_taxas_porte.columns.tolist())
print("Pinto UFs - colunas:", df_pinto_ufs.columns.tolist())


# -------------------------------------------------
# 2. Função auxiliar: taxas por UF e porte
# -------------------------------------------------

def preparar_taxas_por_uf(
    df_taxas_porte: pd.DataFrame,
    df_pinto_ufs: pd.DataFrame,
    especie_col_taxas: str = "scientific_name",
    especie_col_pinto: str = "scientific_name",
    uf_col_pinto: str = "ufs_from_brazilroadkill",
    sep_ufs: str = ";",
    verbose: bool = True,
) -> pd.DataFrame:
    """
    Associa as taxas por espécie/porte (df_taxas_porte) aos estados (UF)
    com base nos locais de estudo do artigo de Pinto (df_pinto_ufs).

    df_taxas_porte  -> mínimo:
        - especie_col_taxas (default: 'scientific_name')
        - 'porte'
        - 'taxa_media_porte'  (ind/dia/100 km)

    df_pinto_ufs    -> mínimo:
        - especie_col_pinto (default: 'scientific_name')
        - uf_col_pinto      (default: 'ufs_from_brazilroadkill', ex: 'MG;SP;RJ')

    Retorna:
        DataFrame com colunas:
        - uf
        - porte
        - taxa_dia_100km   (média das espécies)
        - n_especies
        - taxa_ano_km      (ind/ano/km)
    """
    # Normaliza nomes científicos em df_taxas_porte
    df_tx = df_taxas_porte.copy()
    df_tx[especie_col_taxas] = (
        df_tx[especie_col_taxas]
        .astype(str)
        .str.strip()
        .str.lower()
    )

    # Normaliza nomes e explode UFs em df_pinto_ufs
    df_pinto = df_pinto_ufs.copy()
    df_pinto[especie_col_pinto] = (
        df_pinto[especie_col_pinto]
        .astype(str)
        .str.strip()
        .str.lower()
    )
    df_pinto[uf_col_pinto] = df_pinto[uf_col_pinto].fillna("")
    df_pinto[uf_col_pinto] = df_pinto[uf_col_pinto].astype(str).str.split(sep_ufs)
    df_pinto = df_pinto.explode(uf_col_pinto)
    df_pinto[uf_col_pinto] = df_pinto[uf_col_pinto].str.strip()
    df_pinto = df_pinto[df_pinto[uf_col_pinto] != ""]
    df_pinto = df_pinto.rename(columns={uf_col_pinto: "uf"})

    # Join espécie -> UF
    # Removed .drop(columns=[especie_col_pinto]) as it was removing the 'scientific_name' column itself
    merged = df_tx.merge(
        df_pinto[[especie_col_pinto, "uf"]],
        left_on=especie_col_taxas,
        right_on=especie_col_pinto,
        how="left"
    )

    if verbose:
        total_especies = df_tx[especie_col_taxas].nunique()
        # Ensure the column exists before calling nunique
        especies_com_uf = merged.dropna(subset=["uf"])["scientific_name"].nunique() if "scientific_name" in merged.columns else 0
        print(f"Espécies em df_taxas_porte: {total_especies}")
        print(f"Espécies com pelo menos uma UF em Pinto: {especies_com_uf}")
        if especies_com_uf < total_especies:
            print(
                "Atenção: existem espécies em df_taxas_porte sem correspondência de UF "
                "na tabela de Pinto. Elas não entram na taxa por UF."
            )

    # Agregação por UF e porte
    taxas_uf = (
        merged.dropna(subset=["uf"])
        .groupby(["uf", "porte"], as_index=False)
        .agg(
            taxa_dia_100km=("taxa_media_porte", "mean"),
            n_especies=(especie_col_taxas, "nunique"),
        )
    )

    # Converte dia/100km -> ano/km
    taxas_uf["taxa_ano_km"] = taxas_uf["taxa_dia_100km"] * 365.0 / 100.0

    # Ordena para ficar legível
    taxas_uf = taxas_uf.sort_values(["uf", "porte"]).reset_index(drop=True)
    return taxas_uf


def calcular_taxas_por_uf_extrapoladas(
    df_rodovias: pd.DataFrame,
    df_taxas_porte: pd.DataFrame,
    df_pinto_ufs: pd.DataFrame,
    pav_factor: float = 1.0,
    npav_factor: float = 0.18,
    especie_col_taxas: str = "scientific_name",
    especie_col_pinto: str = "scientific_name",
    uf_col_pinto: str = "ufs_from_brazilroadkill",
    sep_ufs: str = ";",
    verbose: bool = True,
):
    """
    Análise 2: Taxas específicas por UF a partir de Pinto + extrapolação.

    df_rodovias -> mínimo:
        - 'uf'
        - 'km_total_pav'
        - 'km_pav_federal'
        - 'km_total_npav'
        - 'km_npav_federal'

    df_taxas_porte -> ver docstring de preparar_taxas_por_uf.

    Retorna:
        taxas_uf       : taxas derivadas por UF e porte
        resultados_long: extrapolação por UF, porte e tipo_rodovia
    """
    # 1) Derivar taxa por UF e porte
    taxas_uf = preparar_taxas_por_uf(
        df_taxas_porte=df_taxas_porte,
        df_pinto_ufs=df_pinto_ufs,
        especie_col_taxas=especie_col_taxas,
        especie_col_pinto=especie_col_pinto,
        uf_col_pinto=uf_col_pinto,
        sep_ufs=sep_ufs,
        verbose=verbose,
    )

    # 2) Checagem de colunas de rodovias
    df_rod = df_rodovias.copy()
    cols_rod = {"uf", "km_total_pav", "km_pav_federal", "km_total_npav", "km_npav_federal"}
    missing_rod = cols_rod - set(df_rod.columns)
    if missing_rod:
        raise ValueError(f"df_rodovias está faltando colunas: {missing_rod}")

    # 3) Join UF x porte com rodovias
    base = taxas_uf.merge(df_rod, on="uf", how="inner")

    # 4) Estima atropelamentos anuais por tipo de rodovia
    base["atrop_pav_total"]    = base["km_total_pav"]    * base["taxa_ano_km"] * pav_factor
    base["atrop_pav_federal"]  = base["km_pav_federal"]  * base["taxa_ano_km"] * pav_factor
    base["atrop_npav_total"]   = base["km_total_npav"]   * base["taxa_ano_km"] * npav_factor
    base["atrop_npav_federal"] = base["km_npav_federal"] * base["taxa_ano_km"] * npav_factor

    # 5) Formato longo UF × porte × tipo_rodovia
    resultados_long = base.melt(
        id_vars=["uf", "porte", "taxa_dia_100km", "taxa_ano_km"],
        value_vars=[
            "atrop_pav_total",
            "atrop_pav_federal",
            "atrop_npav_total",
            "atrop_npav_federal",
        ],
        var_name="tipo_rodovia",
        value_name="atropelamentos_ano",
    )

    # Nomear tipos de rodovia de forma mais amigável
    mapa_tipos = {
        "atrop_pav_total":    "pav_total",
        "atrop_pav_federal":  "pav_federal",
        "atrop_npav_total":   "npav_total",
        "atrop_npav_federal": "npav_federal",
    }
    resultados_long["tipo_rodovia"] = resultados_long["tipo_rodovia"].map(mapa_tipos)

    # Guardar fatores usados para rastreabilidade
    resultados_long["pav_factor"] = pav_factor
    resultados_long["npav_factor"] = npav_factor

    return taxas_uf, resultados_long


In [ ]:
taxas_uf, resultados_analise2 = calcular_taxas_por_uf_extrapoladas(
    df_rodovias=df_rodovias,
    df_taxas_porte=df_taxas_porte,
    df_pinto_ufs=df_pinto_ufs,
    pav_factor=1.0,
    npav_factor=0.18,
    # aqui já usamos 'scientific_name' como padrão,
    # então não precisa nem passar explicitamente
)

print("\n=== Tabela de taxas por UF (derivadas do artigo) ===")
from IPython.display import display
display(taxas_uf.head())

print("\n=== Tabela de resultados extrapolados (amostra) ===")
display(resultados_analise2.head())


Espécies em df_taxas_porte: 60
Espécies com pelo menos uma UF em Pinto: 47
Atenção: existem espécies em df_taxas_porte sem correspondência de UF na tabela de Pinto. Elas não entram na taxa por UF.

=== Tabela de taxas por UF (derivadas do artigo) ===


,uf,porte,taxa_dia_100km,n_especies,taxa_ano_km
0,AM,grande,17085.706452,1,62362.828548
1,AM,mega,14836.372727,1,54152.760455
2,BA,grande,17085.706452,3,62362.828548
3,BA,mega,14836.372727,1,54152.760455
4,BA,médio,34091.372222,2,124433.508611



=== Tabela de resultados extrapolados (amostra) ===


,uf,porte,taxa_dia_100km,taxa_ano_km,tipo_rodovia,atropelamentos_ano,pav_factor,npav_factor
0,AM,grande,17085.706452,62362.828548,pav_total,4.038679e+08,1.0,0.18
1,AM,mega,14836.372727,54152.760455,pav_total,3.506987e+08,1.0,0.18
2,BA,grande,17085.706452,62362.828548,pav_total,7.907918e+08,1.0,0.18
3,BA,mega,14836.372727,54152.760455,pav_total,6.866841e+08,1.0,0.18
4,BA,médio,34091.372222,124433.508611,pav_total,1.577879e+09,1.0,0.18


In [ ]:
taxas_uf.to_csv('analise2_taxas_por_uf.csv', index=False)
resultados_analise2.to_csv('analise2_resultados_extrapolados.csv', index=False)

from google.colab import files
files.download('analise2_taxas_por_uf.csv')
files.download('analise2_resultados_extrapolados.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# --- Process analise2_taxas_por_uf.csv ---

# Read the generated CSV file
df_taxas_uf = pd.read_csv('analise2_taxas_por_uf.csv')

# Create a new Word Document
document_taxas_uf = Document()

document_taxas_uf.add_heading('Tabela de Taxas por UF', level=1)

# Add a table to the document
table_taxas_uf = document_taxas_uf.add_table(rows=1, cols=len(df_taxas_uf.columns))
table_taxas_uf.autofit = True

# Set the column headers
for i, col_name in enumerate(df_taxas_uf.columns):
    table_taxas_uf.rows[0].cells[i].text = col_name

# Add the data rows
for index, row in df_taxas_uf.iterrows():
    cells = table_taxas_uf.add_row().cells
    for i, cell_value in enumerate(row):
        cells[i].text = str(cell_value)

# Define the filename
output_filename_taxas_uf = 'analise2_taxas_por_uf.docx'

# Save the document
document_taxas_uf.save(output_filename_taxas_uf)

# Download the file
files.download(output_filename_taxas_uf)

print(f"Documento '{output_filename_taxas_uf}' gerado e pronto para download.")

# --- Process analise2_resultados_extrapolados.csv ---

# Read the generated CSV file
df_resultados_extrapolados = pd.read_csv('analise2_resultados_extrapolados.csv')

# Create a new Word Document
document_resultados_extrapolados = Document()

document_resultados_extrapolados.add_heading('Tabela de Resultados Extrapolados da Análise 2', level=1)

# Add a table to the document
table_resultados_extrapolados = document_resultados_extrapolados.add_table(rows=1, cols=len(df_resultados_extrapolados.columns))
table_resultados_extrapolados.autofit = True

# Set the column headers
for i, col_name in enumerate(df_resultados_extrapolados.columns):
    table_resultados_extrapolados.rows[0].cells[i].text = col_name

# Add the data rows
for index, row in df_resultados_extrapolados.iterrows():
    cells = table_resultados_extrapolados.add_row().cells
    for i, cell_value in enumerate(row):
        cells[i].text = str(cell_value)

# Define the filename
output_filename_resultados_extrapolados = 'analise2_resultados_extrapolados.docx'

# Save the document
document_resultados_extrapolados.save(output_filename_resultados_extrapolados)

# Download the file
files.download(output_filename_resultados_extrapolados)

print(f"Documento '{output_filename_resultados_extrapolados}' gerado e pronto para download.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Documento 'analise2_taxas_por_uf.docx' gerado e pronto para download.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Documento 'analise2_resultados_extrapolados.docx' gerado e pronto para download.


In [ ]:
total_brasil = resultados_analise2["atropelamentos_ano"].sum()
total_brasil


np.float64(47616483771.5912)